In [ ]:
import os.path

import numpy as np
import numpy.linalg as la
import matplotlib
import matplotlib.pyplot as plt
import netCDF4 as nc4

import bin_model as bm

# Time convergence plots

In [ ]:
KERNEL_FILE_NAME = "kernels/Hall_ScottChen_kernel_nb336.nc"
EXP_FILE_NAME_TEMPLATE = \
    os.path.join("convergence_experiments",
                 "time_convergence_experiment_dtexp{}.nc")

In [ ]:
END_TIME = 3600

NDT = 10
dtexps = list(range(NDT))

In [ ]:
#with nc4.Dataset(KERNEL_FILE_NAME, "r") as nc:
#    netcdf_file = bm.NetcdfFile(nc)
#    _, kernel, _, ktens = netcdf_file.read_cgk()

In [ ]:
reference_file_name = EXP_FILE_NAME_TEMPLATE.format(dtexps[-1])
with nc4.Dataset(reference_file_name, "r") as nc:
    netcdf_file = bm.NetcdfFile(nc)
    exp = netcdf_file.read_full_experiment([]) # don't need ([ktens])

In [ ]:
def get_dsd_values_for_comparison(exp, dtexp):
    nb = exp.mass_grid.num_bins
    ncompare = int(END_TIME // 10)
    stride = 2**dtexp
    output = np.zeros((ncompare, nb))
    for i in range(ncompare):
        output[i,:] = exp.states[stride*i].dsd()
    return output

In [ ]:
def get_moment_values_for_comparison(n, exp, dtexp):
    ncompare = int(END_TIME // 10)
    stride = 2**dtexp
    output = np.zeros((ncompare,))
    for i in range(ncompare):
        output[i] = exp.states[stride*i].dsd_moment(n)
    return output

In [ ]:
reference_dsd_values = get_dsd_values_for_comparison(exp, dtexps[-1])
reference_mom0s = get_moment_values_for_comparison(0, exp, dtexps[-1])
reference_mom6s = get_moment_values_for_comparison(6, exp, dtexps[-1])
reference_dbzs = 10. * np.log10(reference_mom6s)

In [ ]:
rmse = np.zeros((NDT-1,))
mom0_max_errs = np.zeros((NDT-1,))
mom6_max_errs = np.zeros((NDT-1,))
dbz_max_errs = np.zeros((NDT-1,))
for i in range(NDT-1):
    dtexp = dtexps[i]
    exp_file_name = EXP_FILE_NAME_TEMPLATE.format(dtexp)
    with nc4.Dataset(exp_file_name, "r") as nc:
        netcdf_file = bm.NetcdfFile(nc)
        exp = netcdf_file.read_full_experiment([]) # don't need ([ktens])
    dsd_values = get_dsd_values_for_comparison(exp, dtexp)
    rmse[i] = la.norm(dsd_values - reference_dsd_values)
    mom0s = get_moment_values_for_comparison(0, exp, dtexp)
    mom6s = get_moment_values_for_comparison(6, exp, dtexp)
    dbzs = 10. * np.log10(mom6s)
    mom0_max_errs[i] = np.abs(mom0s - reference_mom0s).max()
    mom6_max_errs[i] = np.abs(mom6s - reference_mom6s).max()
    dbz_max_errs[i] = np.abs(dbzs - reference_dbzs).max()

In [ ]:
dts = np.array([10. * 2**(-x) for x in dtexps])
plt.loglog(dts[:-1], rmse, color='k', marker='o')
plt.loglog(dts[-6:-1], dts[-6:-1]**5 * (rmse[-3] / dts[-4]**5),
           color='k', linestyle='--')

In [ ]:
dts = np.array([10. * 2**(-x) for x in dtexps])
plt.loglog(dts[:-1], mom0_max_errs, color='k', marker='o')
plt.loglog(dts[-6:-1], dts[-6:-1]**5 * (mom0_max_errs[-3] / dts[-4]**5),
           color='k', linestyle='--')

In [ ]:
dts = np.array([10. * 2**(-x) for x in dtexps])
plt.loglog(dts[:-1], mom6_max_errs, color='k', marker='o')
plt.loglog(dts[-6:-1], dts[-6:-1]**5 * (mom6_max_errs[-3] / dts[-4]**5),
           color='k', linestyle='--')

In [ ]:
dts = np.array([10. * 2**(-x) for x in dtexps])
plt.loglog(dts[:-1], dbz_max_errs, color='k', marker='o')
plt.loglog(dts[-6:-1], dts[-6:-1]**5 * (dbz_max_errs[-3] / dts[-4]**5),
           color='k', linestyle='--')

# Mass convergence plots

In [ ]:
KERNEL_FILE_NAME_TEMPLATE = \
    os.path.join("kernels", "Hall_ScottChen_kernel_nb{}.nc")

EXP_FILE_NAME_TEMPLATE = \
    os.path.join("convergence_experiments",
                 "mass_convergence_experiment_nb{}.nc")

NNB = 7

nbs = np.array([42 * 2**i for i in range(NNB)])

In [ ]:
#reference_kernel_file_name = KERNEL_FILE_NAME_TEMPLATE.format(nbs[-1])
#with nc4.Dataset(reference_kernel_file_name, "r") as nc:
#    netcdf_file = bm.NetcdfFile(nc)
#    _, kernel, _, ktens = netcdf_file.read_cgk()

In [ ]:
reference_exp_file_name = EXP_FILE_NAME_TEMPLATE.format(nbs[-1])
with nc4.Dataset(reference_exp_file_name, "r") as nc:
    netcdf_file = bm.NetcdfFile(nc)
    exp = netcdf_file.read_full_experiment([])# don't need ([ktens])

In [ ]:
def get_dsd_values_for_comparison(exp, lowest_nb):
    nt = exp.num_time_steps
    nb = exp.mass_grid.num_bins
    ncompare = lowest_nb
    stride = int(nb // lowest_nb)
    output = np.zeros((nt, ncompare))
    for i in range(nt):
        output[i,:] = exp.states[i].dsd()[::stride]
    return output

In [ ]:
def get_moment_values_for_comparison(n, exp):
    nt = exp.num_time_steps
    output = np.zeros((nt,))
    for i in range(nt):
        output[i] = exp.states[i].dsd_moment(n)
    return output

In [ ]:
reference_dsd_values = get_dsd_values_for_comparison(exp, nbs[0])
reference_mom0s = get_moment_values_for_comparison(0, exp)
reference_mom6s = get_moment_values_for_comparison(6, exp)
reference_dbzs = 10. * np.log10(reference_mom6s)

In [ ]:
rmse = np.zeros((NNB-1,))
mom0_max_errs = np.zeros((NNB-1,))
mom6_max_errs = np.zeros((NNB-1,))
dbz_max_errs = np.zeros((NNB-1,))
for i in range(NNB-1):
    nb = nbs[i]
    kernel_file_name = KERNEL_FILE_NAME_TEMPLATE.format(nb)
    #with nc4.Dataset(kernel_file_name, "r") as nc:
    #    netcdf_file = bm.NetcdfFile(nc)
    #    _, kernel, _, ktens = netcdf_file.read_cgk()
    exp_file_name = EXP_FILE_NAME_TEMPLATE.format(nb)
    with nc4.Dataset(exp_file_name, "r") as nc:
        netcdf_file = bm.NetcdfFile(nc)
        exp = netcdf_file.read_full_experiment([]) # don't need ([ktens])
    dsd_values = get_dsd_values_for_comparison(exp, nbs[0])
    rmse[i] = la.norm(dsd_values - reference_dsd_values)
    mom0s = get_moment_values_for_comparison(0, exp)
    mom6s = get_moment_values_for_comparison(6, exp)
    dbzs = 10. * np.log10(mom6s)
    mom0_max_errs[i] = np.abs(mom0s - reference_mom0s).max()
    mom6_max_errs[i] = np.abs(mom6s - reference_mom6s).max()
    dbz_max_errs[i] = np.abs(dbzs - reference_dbzs).max()

In [ ]:
plt.loglog(nbs[:-1], rmse, color='k', marker='o')
plt.loglog(nbs[:-1], (1./nbs[:-1]) * (rmse[2] * nbs[2]),
           color='k', linestyle='--')

In [ ]:
plt.loglog(nbs[:-1], mom0_max_errs, color='k', marker='o')
plt.loglog(nbs[:-1], (1./nbs[:-1]) * (mom0_max_errs[2] * nbs[2]),
           color='k', linestyle='--')

In [ ]:
plt.loglog(nbs[:-1], mom6_max_errs, color='k', marker='o')
plt.loglog(nbs[:-1], (1./nbs[:-1]) * (mom6_max_errs[2] * nbs[2]),
           color='k', linestyle='--')

In [ ]:
plt.loglog(nbs[:-1], dbz_max_errs, color='k', marker='o')
plt.loglog(nbs[:-1], (1./nbs[:-1]) * (dbz_max_errs[2] * nbs[2]),
           color='k', linestyle='--')